In [1]:
from torch import  nn


## 3.1.1 卷积层
- 计算过程
    - 书上的是滑动step后每个元素进行相乘
    - pytorch中是非常简单的


In [2]:
# 快速搭建卷积层
conv = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,stride=1,padding=1,dilation=1,groups=1,bias=True)  

# 查看卷积核的基本信息，本质上是一个Module
conv


Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [3]:
# 通过.weight和.bias查看卷积核的权重和偏置
conv.weight

Parameter containing:
tensor([[[[ 0.0377,  0.2636, -0.2682],
          [-0.1179,  0.3013,  0.2442],
          [ 0.3137, -0.0469, -0.1935]]]], requires_grad=True)